In [1]:
#%reset
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")

import torch
import numpy as np
import matplotlib.pyplot as plt

import implicitmodules.torch as dm

torch.set_default_tensor_type(torch.DoubleTensor)

In [2]:
def rot(theta):
    return torch.tensor([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])

width = 25.
height = 3.
nx = 80
ny = 8
x, y = torch.meshgrid([torch.linspace(-width/2, width/2, nx), torch.linspace(-height/2, height/2, ny)])
pts_sil = dm.usefulfunctions.grid2vec(x, y)
ps = torch.zeros_like(pts_sil)

width1 = 20.
height1 = 2.
nx1 = 30
ny1 = 3
x1, y1 = torch.meshgrid([torch.linspace(-width1/2, width1/2, nx1), torch.linspace(-height1/2, height1/2, ny1)])
pts = dm.usefulfunctions.grid2vec(x1, y1)
R = rot(0.).repeat(pts.shape[0], 1, 1)
p1 = torch.zeros_like(pts)
p1[0, 0] = -15000.

C = torch.zeros(pts.shape[0], 2, 1)
C[:, 0] = pts[:, 1].view(-1, 1)*torch.cos(np.pi*pts[:, 0].view(-1, 1)/width1*2*4)

sigma = 1.1
nu = 0.

model = dm.implicitmodules.ImplicitModule1(dm.manifold.Stiefel(2, pts.shape[0]), C, sigma, nu)
model.manifold.fill_gd((pts.view(-1).requires_grad_(), R.view(-1).requires_grad_()))
model.manifold.fill_cotan((p1.view(-1).requires_grad_(), torch.zeros_like(R).view(-1).requires_grad_()))

sil = dm.deformationmodules.SilentPoints(dm.manifold.Landmarks(2, pts_sil.shape[0]))
sil.manifold.fill_gd(pts_sil.view(-1).requires_grad_())
sil.manifold.fill_cotan(ps.view(-1).requires_grad_())

compound = dm.deformationmodules.CompoundModule([sil, model])
out = dm.shooting.shoot(dm.hamiltonian.Hamiltonian(compound), it=10, method='torch_euler')

AttributeError: module 'implicitmodules.torch' has no attribute 'usefulfunctions'

In [ ]:
aabb = dm.usefulfunctions.AABB.build_from_points(torch.cat([compound[0].manifold.gd.view(-1, 2).detach(), pts_sil], dim=0))
aabb.squared()
%matplotlib qt5
ax = plt.subplot(1, 2, 1)
plt.plot(pts_sil.numpy()[:, 0], pts_sil.numpy()[:, 1], '.')
plt.plot(pts.numpy()[:, 0], pts.numpy()[:, 1], 'x')
x, y = dm.usefulfunctions.vec2grid(pts_sil, nx, ny)
dm.usefulfunctions.plot_grid(ax, x.numpy(), y.numpy(), color='C0')
plt.axis(aabb.get_list())

ax1 = plt.subplot(1, 2, 2)
plt.plot(compound[0].manifold.gd.detach().view(-1, 2).numpy()[:, 0], compound[0].manifold.gd.detach().view(-1, 2).numpy()[:, 1], '.')
plt.plot(compound[1].manifold.gd[0].detach().view(-1, 2).numpy()[:, 0], compound[1].manifold.gd[0].detach().view(-1, 2).numpy()[:, 1], 'x')
x, y = dm.usefulfunctions.vec2grid(compound[0].manifold.gd.view(-1, 2), nx, ny)
dm.usefulfunctions.plot_grid(ax1, x.detach().numpy(), y.detach().numpy(), color='C0')
plt.axis(aabb.get_list())
plt.show()